## Quick Start Guide

`nimCSO` is a high-performance tool implementing several methods for selecting components (data dimensions) in compositional datasets, which optimize the data availability and density for applications such as machine learning. Making said choice is a combinatorically hard problem for complex compositions existing in high-dimensional spaces due to the interdependency of components being present. 

It is an interdisciplinary tool applicable to any field where data is composed of a large number of independent components and their interaction is of interest in a modeling effort, including some everyday contexts. For instance, *"Given 100 spices at the supermarket, which 20, 30, or 40 should I stock in my pantry to maximize the number of unique dishes I can spice according to recipe?"*. Critically, this is not as simple as frequency-based selection because, e.g., *removing* less common nutmeg and cinnamon from your shopping list will *prevent* many recipes with the frequent vanilla, but won't affect those using black pepper.

The purpose of this guide is to demonstrate some common use cases of `nimCSO` and go in a bit more into the details of how it could be used, but it is not by any means extensive. If something is not covered but you would like to see it here, please do not heasitate to open an issue on GitHub and let us know!

### Dataset, Config, and Compilation
To get started, let's first recap what we need to do to get `nimCSO` up and running.

**1.** Install nim and dependencies, but **that's already done for you if you are in the Codespace!**. You can see what was run to get the environment set up in the [`Dockerfile`](../.devcontainer/Dockerfile).

**2.** Create the dataset. For now, let's just use the example one (based on the [ULTERA Database](https://ultera.org)) that comes with the package. Relative to this notebook, the dataset is located at `../dataList.txt`. Let's have a look at the first few lines of the file to see what it looks like. For details, please consult the main `README` or online documentation.

In [18]:
!head -n 8 ../dataList.txt

Al,Co,Cr,Cu,Fe,Ni
Nb,Ta,Ti
Co,Cr,Ni
Al,Co,Cr,Fe,Mn,Ni
Al,Co,Fe,Mo,Ni
Hf,Nb,Ti,V
Co,Cr,Fe,Nb,Ni
Al,Co,Cr,Cu,Fe,Ni


**3.** Now, we need to create task `config.yaml` file that will describe what we are doing and point to our data file. That was already done for you in the [`config.yaml`](config.yaml) file, but you are more than welcome to play and modify it.

**4.** Finally, we can run the `nimCSO` package to get the results. To do so, we will use one long command you can see below. Let's break it down:
- `!` is a Jupyter Notebook magic command that allows us to run shell commands from within the notebook.

- `nim` is the official Nim language compiler. 

- `c` instructs `nim` compiler to use `C` compiler to optimize and compile intermediate code. You can also use `cpp` to use `C++` compiler or `objc` to use `Objective-C` compiler. If you want, you can also compile directly with LLVM using [`nlvm`](https://github.com/arnetheduck/nlvm), but it isn't pre-installed for you here.

- `-f` is a flag to force the compiler to compile everything, even if the code didn't change. We want this because `config.yaml`, which tells `nimCSO` how to write itself, is not tracked by the compiler, but is critical to the compilation process (see two point below).

- `-d:release` is a flag that tells the compiler to optimize the code for release. You can also use `-d:debug` to compile the code with better debugging support, but it will be slower and it will not prevent bugs from happening. There is also `-d:danger` that will disable all runtime checks and run a bit faster, but you no longer get memory safety guarantees.

- `-d:configPath=config.yaml` is a flag pointing to **`config.yaml` that is read and tells `nimCSO` (not the compiler!) how to write itself *before* the compilation starts.** That's the magic metaprogramming sauce enabling us to write functions which `C`/`C++` compiler can then turn into single deterministically allocated and exectuted machine code through [inlining](https://en.wikipedia.org/wiki/Inline_expansion).

- `out:nimcso` is just telling the compiler to output the compiled binary right here and name it `nimcso`. You can name it whatever you want, but it's a good idea to name it something that makes sense.

- `../src/nimcso` is pointing to the source code of `nimCSO` package to compile, relative to this notebook.

Let's run the command and see what happens! Shouldn't take more than a few seconds.

In [19]:
!nim c -f -d:release -d:configPath=config.yaml --out:nimcso ../src/nimcso 

Hint: used config file '/opt/conda/nim/config/nim.cfg' [Conf]
Hint: used config file '/opt/conda/nim/config/config.nims' [Conf]
...........................................................................................................................................................
/root/.nimble/pkgs2/nimblas-0.3.0-d5033749759fc7a2a316acf623635dcb6d69d32a/nimblas/private/common.nim(52, 7) Hint: Using BLAS library with name: lib(blas|cblas|openblas).so(||.3|.2|.1|.0) [User]
...........................................................................
config.yaml
CC: ../../../opt/conda/nim/lib/system/exceptions.nim
CC: ../../../opt/conda/nim/lib/std/private/digitsutils.nim
CC: ../../../opt/conda/nim/lib/std/assertions.nim
CC: ../../../opt/conda/nim/lib/system/dollars.nim
CC: ../../../opt/conda/nim/lib/std/syncio.nim
CC: ../../../opt/conda/nim/lib/system.nim
CC: ../../../opt/conda/nim/lib/pure/parseutils.nim
CC: ../../../opt/conda/nim/lib/pure/math.nim
CC: ../../../opt/conda/nim/lib/pure/u

Now, let's run `nimCSO` and see what happens!

In [20]:
!./nimcso

Using 1 uint64s to store 19 elements.
Configured for task: QuickStart (Just a copy of RCCA Palette from Senkov 2018 Review)
***** nimCSO (Composition Space Optimization) *****
To use from command line, provide parameters. Currently supported usage:

--covBenchmark    | -cb   --> Run small coverage benchmarks under two implementations.
--expBenchmark    | -eb   --> Run small node expansion benchmarks.
--leastPreventing | -lp   --> Run a search for single-elements preventing the least data, i.e. the least common elements.
--mostCommon      | -mc   --> Run a search for most common elements.
--bruteForce      | -bf   --> Run brute force algorithm after getting ETA. Note that it is not feasible for more than 25ish elements.
--bruteForceInt   | -bfi  --> Run brute force algorithm with faster but not extensible uint64 representation after getting ETA. Up to 64 elements only.
--geneticSearch   | -gs   --> Run a genetic search algorithm.
--algorithmSearch | -as   --> Run a custom problem-inform

You should have seen a neat concise `help` message that tells you how to use `nimCSO`. Let's start with a "coverage" benchmark to see how fast can we check how many datapoints will be prevented (will have to be removed from the original dataset) if we remove the first 5 elements of `elementOrder`. By our earlier analogy to selecting spices in the supermarket, this corresponds to checking how many recipies you won't be able to properly spice if you decide not to buy the first 5 spices on your shopping list.

In [4]:
!./nimcso -cb

Using 1 uint64s to store 19 elements.
Configured for task: QuickStart (Just a copy of RCCA Palette from Senkov 2018 Review)
***** nimCSO (Composition Space Optimization) *****
Running coverage benchmark with int8 Tensor representation
Tensor[system.int8] of shape "[1, 19]" on backend "Cpu"
|1      1     1     1     1     1     0     0     0     0     0     0     0     0     0     0     0     0     0|
CPU Time [arraymancer+randomizing] 133.6μs
Prevented count:995

Running coverage benchmark with BitArray representation
CPU Time [bitty+randomizing] 13.6μs
    | 1| 2| 3| 4| 5| 6| 7| 8| 9|10|11|12|13|14|15|16|17|18|19|
 19 | 1| 1| 1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
MoTaVWTiZr->995
Prevented count:995

Running coverage benchmark with bool arrays representation (BitArray graph retained)
CPU Time [bit&boolArrays+randomizing] 16.1μs
    | 1| 2| 3| 4| 5| 6| 7| 8| 9|10|11|12|13|14|15|16|17|18|19|
 19 | 1| 1| 1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
MoTaVWTiZr->995
Preve

### Key Routines and Brute Forcing

And if you were able to run that, you are all set to start using `nimCSO`!

Let's try the simplest routine `mostCommon` or *What are the most common elements in the dataset?*

In [5]:
!./nimcso --mostCommon

Using 1 uint64s to store 19 elements.
Configured for task: QuickStart (Just a copy of RCCA Palette from Senkov 2018 Review)
***** nimCSO (Composition Space Optimization) *****

Running search for single-elements preventing the most data.
 0: Cr->667
 1: Ti->649
 2: Fe->622
 3: Ni->620
 4: Nb->587
 5: Co->573
 6: Al->569
 7: Mo->466
 8: Zr->346
 9: Ta->330
10: V->256
11: Hf->219
12: W->207
13: Si->92
14: B->69
15: Re->55
16: C->36
17: Y->3
18: N->1

nimCSO Done!


If you didn't modify anything, you should now see that elements like `N`, `Y`, `C`, and `Re`, are not very common in the dataset, while `Cr`, `Ti`, `Fe`, and `Ni` are very common. When it comes to them, it's pretty obvious that removing the first group will be the first choice, while the latter will be the last, if we want to keep the dataset as extensive as possible.

The critical question here is, *which of the intermediate elements like `Hf`, `V`, `Ta`, or `Zr` should we remove first?*

With a dataset spanning 19 elements, the solution space is around 0.5M, so we can actually just brute force it in seconds :)

In [6]:
!./nimcso -bfi

Using 1 uint64s to store 19 elements.
Configured for task: QuickStart (Just a copy of RCCA Palette from Senkov 2018 Review)
***** nimCSO (Composition Space Optimization) *****

Running brute force algorithm for 19 elements and 1349 data points.
Solution space size: 524287
Task ETA Estimate: 7 seconds and 30 milliseconds
 0: ->0
 1: N->1
 2: YN->4
 3: YCN->39
 4: ReYCN->89
 5: ReYBCN->142
 6: SiReYBCN->203
 7: WSiReYBCN->340
 8: WHfSiReYBCN->511
 9: TaWHfSiReYBCN->630
10: TaWZrHfSiReYBCN->735
11: TaVWZrHfSiReYBCN->816
12: TaVWZrHfNbSiReYBCN->859
13: TaVWTiZrHfNbSiReYBCN->952
14: MoTaVWTiZrHfNbSiReYBCN->1038
15: MoTaVWTiZrHfNbAlSiReYBCN->1304
16: TaVWTiZrHfNbCrAlCoNiReFeYCN->1327
17: MoTaVWTiZrHfNbCrAlSiCoNiReFeYB->1349
18: MoTaVWTiZrHfNbCrAlSiCoNiReFeYBC->1349
19: MoTaVWTiZrHfNbCrAlSiCoNiReFeYBCN->1349
CPU Time [Brute Force] 7308.5ms

nimCSO Done!


Let's look at the result! As expected, `N`, `Y`, `C`, and `Re` are removed first (0-4) and then the trend follows for a bit to `Hf` **The first break is `V`, you can notice that it's better to remove either or both `Ta` or `Zr` first, despite the fact that they are nearly 50% more common than `V`!** That's because they often coocur with `Re` and `Hf`, which are not common.

By our earlier analogy to selecting spices in the supermarket, we already removed nutmeg and cinnamon from our shopping list so if we want to maximize the number of recipies we can fulfill, it's better to buy black pepper than more common vanilla.

We can test exactly how much more data we will have if we remove `Ta` insead of `V` by using the `--singleSolution` / `-ss` routine.

In [10]:
!./nimcso -ss Ta W Hf Si Re Y B C N -ss V W Hf Si Re Y B C N

Using 1 uint64s to store 19 elements.
Configured for task: QuickStart (Just a copy of RCCA Palette from Senkov 2018 Review)
***** nimCSO (Composition Space Optimization) *****
Testing solution with @[@["Ta", "W", "Hf", "Si", "Re", "Y", "B", "C", "N"], @["V", "W", "Hf", "Si", "Re", "Y", "B", "C", "N"]]
 9: TaWHfSiReYBCN->630
 9: VWHfSiReYBCN->697

nimCSO Done!


Wow! Looking at the `--mostCommon` output from earlier, we can see that **`Ta` is present in 74 more datapoints than `V`, but after removing `WHfSiReYBCN`, picking `V` as one of 10 elements to model will result in 67 *more* datapoints.** Relative to a dataset without interdependencies, that's a 141 datapoint difference!

And another case that breaks from the ordering is `Mo`, which is better to keep than much more common `Nb`, and after `Nb` is removed, even better thank keeping the `Ti`, which is the second most common element in the dataset!

Similarly to what we did with `V` vs. `Ta`, we can test how much more data we will have if we remove `Nb` instead of `Mo` by using the `--singleSolution` / `-ss` routine.

In [12]:
!./nimcso -ss Ta V W Zr Hf Nb Si Re Y B C N -ss Ta V W Zr Hf Mo Si Re Y B C N -ss Ta V W Zr Hf Ti Si Re Y B C N

Using 1 uint64s to store 19 elements.
Configured for task: QuickStart (Just a copy of RCCA Palette from Senkov 2018 Review)
***** nimCSO (Composition Space Optimization) *****
Testing solution with @[@["Ta", "V", "W", "Zr", "Hf", "Nb", "Si", "Re", "Y", "B", "C", "N"], @["Ta", "V", "W", "Zr", "Hf", "Mo", "Si", "Re", "Y", "B", "C", "N"], @["Ta", "V", "W", "Zr", "Hf", "Ti", "Si", "Re", "Y", "B", "C", "N"]]
12: TaVWZrHfNbSiReYBCN->859
12: MoTaVWZrHfSiReYBCN->935
12: TaVWTiZrHfSiReYBCN->938

nimCSO Done!


We can see that **`Nb` is present in 121 more datapoints than `Mo`, but after removing `TaVWZrHfSiReYBCN`, picking `Mo` as one of 7 elements to model will result in 76 *more* datapoints.** Relative to a dataset without interdependencies, that's a 197 datapoint difference, which is even more than the `Ta` vs. `V` case! Additionally, we can see that `Ti` is only 3 datapoints better than `Mo`, despite being present in 183 more datapoints than `Mo`.

### Algorithm Search

The `--bruteForceInt`/`-bfi` routine we used to find the solutions worked great for our 19-element dataset and took only a few seconds on the low-performance Codespace machine, but in many cases dimensionality of the problem will be too high to brute force it.

Let's now try to use the `--algorithmSearch`/`-as` routine, which takes advantage of some assumptions known to be valid or likely to be valid (see manuscript), to limit the search space and find the solution in a reasonable time. Let's try it now!

In [16]:
!./nimcso -as

Using 1 uint64s to store 19 elements.
Configured for task: QuickStart (Just a copy of RCCA Palette from Senkov 2018 Review)
***** nimCSO (Composition Space Optimization) *****

Running Algorithm Search for 19 elements.
 1: N->1  (tree size: 19)
 2: YN->4  (tree size: 52)
 3: YCN->39  (tree size: 113)
 4: ReYCN->89  (tree size: 275)
 5: ReYBCN->142  (tree size: 581)
 6: SiReYBCN->203  (tree size: 690)
 7: WSiReYBCN->340  (tree size: 1818)
 8: WHfSiReYBCN->511  (tree size: 3873)
 9: TaWHfSiReYBCN->630  (tree size: 5213)
10: TaWZrHfSiReYBCN->735  (tree size: 4833)
11: TaVWZrHfSiReYBCN->816  (tree size: 4192)
12: TaVWZrHfNbSiReYBCN->859  (tree size: 3784)
13: TaVWTiZrHfNbSiReYBCN->952  (tree size: 2955)
14: MoTaVWTiZrHfNbSiReYBCN->1038  (tree size: 1765)
15: MoTaVWTiZrHfNbAlSiReYBCN->1304  (tree size: 45)
16: MoTaVWTiZrHfNbAlSiReFeYBCN->1338  (tree size: 8)
17: MoTaVWTiZrHfNbCrAlSiReFeYBCN->1349  (tree size: 4)
18: MoTaVWTiZrHfNbCrAlSiNiReFeYBCN->1349  (tree size: 4)
CPU Time [exploring] 1

As you can see, **the algorithm reproduced the same results as the brute force search around 100 times faster**, except for third-to-last step because dataset had points with at least 3 elements breaking its backtracking assumptions.

### Genetic Search

For cases where the dimensionality of the problem is too high to either brute-force or use the algorithm search, we can still use the `--geneticSearch`/`-gs` routine to find the solution in a reasonable time. Let's try it now!

In [17]:
!./nimcso -gs

Using 1 uint64s to store 19 elements.
Configured for task: QuickStart (Just a copy of RCCA Palette from Senkov 2018 Review)
***** nimCSO (Composition Space Optimization) *****

Running Genetic Search algorithm for 19 elements and 1349 data points.
Initiating each level with 100 random solutions and expanding 100 solutions at each level for up to 1000 iterations.
 1: N->1
 2: YN->4  (queue size: 256)
 3: YCN->39  (queue size: 615)
 4: ReYCN->89  (queue size: 869)
 5: ReYBCN->142  (queue size: 929)
 6: SiReYBCN->203  (queue size: 1379)
 7: WSiReYBCN->340  (queue size: 1267)
 8: WHfSiReYBCN->511  (queue size: 1631)
 9: TaWHfSiReYBCN->630  (queue size: 1578)
10: TaWZrHfSiReYBCN->735  (queue size: 1835)
11: TaVWZrHfSiReYBCN->816  (queue size: 1621)
12: TaVWZrHfNbSiReYBCN->859  (queue size: 1746)
13: VWCrAlSiCoNiReFeYBCN->1176  (queue size: 1713)
14: MoTaVWTiZrHfNbSiReYBCN->1038  (queue size: 1565)
15: MoTaVWCrAlSiCoNiReFeYBCN->1320  (queue size: 1028)
16: TaVWTiZrHfNbCrAlCoNiReFeYCN->1327  

Please note that the results are stochastic, so you might get different results than ones shown below if you run the command again.

## Summary

Now, you should be able to apply `nimCSO` to your own dataset and get some valuable insights on how to model it!

If you are working in a Codespace, you can just do everything right in this notebook by simply modifying the `config.yaml` file and running the commands you just learned about. The Codespace will be persisted until you explicitly delete it, so you can come back to it later and continue your work by clicking on the link in the "Open in Codespaces" badge in the README of the repository and resuming your work.